Object Detection for Image datasets


In [1]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 857.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.6/871.6 kB 9.5 MB/s eta 0:00:00


In [2]:
import os

# Create a main folder to store everything
livestock_count = '/content/livestock_detection'
os.makedirs(livestock_count, exist_ok=True)

# Create subfolders for datasets, results, and models
os.makedirs(f'{livestock_count}/datasets', exist_ok=True)
os.makedirs(f'{livestock_count}/results', exist_ok=True)
os.makedirs(f'{livestock_count}/models', exist_ok=True)

print("Folders created successfully.")


Folders created successfully.


In [3]:
# List contents of the main folder
!ls /content/livestock_detection

datasets  models  results


In [5]:
from google.colab import files

# Upload files to the 'datasets' folder
uploaded = files.upload()

# Save uploaded files to the dataset folder
for filename in uploaded.keys():
    with open(os.path.join(f'{livestock_count}/datasets', filename), 'wb') as f:
        f.write(uploaded[filename])

Saving goat.jpg to goat.jpg
Saving camel.jpg to camel.jpg
Saving hipoimg.jpg to hipoimg.jpg


In [6]:
# List contents of the 'datasets' folder
!ls /content/livestock_detection/datasets

camel.jpg  goat.jpg  hipoimg.jpg


In [7]:
from google.colab import files

# Download a file from the folder
files.download('/content/livestock_detection/datasets/hipoimg.jpg')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
from ultralytics import YOLO

# Load a pre-trained YOLOv8 model (you can choose 'yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', or 'yolov8x')
model = YOLO('yolov8n.pt')  # Using 'yolov8n' for a faster but smaller model


100%|██████████| 6.25M/6.25M [00:00<00:00, 76.0MB/s]


In [9]:
import os
from PIL import Image

# Path to an image from your dataset folder
img_path = '/content/livestock_detection/datasets/goat.jpg'

# Run object detection on the image using YOLOv8 model
results = model(img_path)

# Display the detection results (from the first element in the results list)
results[0].plot()

# Save the detection plot manually to the 'results' folder
save_dir = '/content/livestock_detection/results'
os.makedirs(save_dir, exist_ok=True)  # Ensure the results folder exists

# The plot method returns a numpy array, which we need to convert to an image using PIL
result_img = results[0].plot(show=False)  # 'show=False' avoids showing the image inline

# Convert numpy array to an image and save it using PIL
result_img_pil = Image.fromarray(result_img)  # Convert numpy array to PIL image
result_img_pil.save(f"{save_dir}/detection_result.jpg")  # Save the image

print(f"Results saved to: {save_dir}/detection_result.jpg")



image 1/1 /content/livestock_detection/datasets/goat.jpg: 448x640 6 sheeps, 366.3ms
Speed: 20.5ms preprocess, 366.3ms inference, 36.3ms postprocess per image at shape (1, 3, 448, 640)
Results saved to: /content/livestock_detection/results/detection_result.jpg


In [10]:
# List the saved results
!ls /content/livestock_detection/results

detection_result.jpg


In [11]:
from google.colab import files

# Path to the saved image
result_image_path = '/content/livestock_detection/results/detection_result.jpg'

# Download the image
files.download(result_image_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Object Detection for Video datasets

In [23]:
!pip install ultralytics

In [32]:
from google.colab import files
import os

# Define the path for datasets
livestock_count = '/content/livestock_detection'
datasets_folder = os.path.join(livestock_count, 'datasets')

# Create necessary directories
os.makedirs(datasets_folder, exist_ok=True)

# Step 1: Upload the video file
def upload_video():
    uploaded = files.upload()

    video_path = None
    for filename in uploaded.keys():
        video_path = os.path.join(datasets_folder, filename)
        with open(video_path, 'wb') as f:
            f.write(uploaded[filename])

    if video_path and os.path.exists(video_path):
        print(f"Video uploaded successfully: {video_path}")
    else:
        print("Error: Video file not found.")

    return video_path

# Upload the video and get the path
video_path = upload_video()


Saving livestock_video_dataset(1).mp4 to livestock_video_dataset(1).mp4
Video uploaded successfully: /content/livestock_detection/datasets/livestock_video_dataset(1).mp4


In [33]:
import os

# Check if video file exists
if os.path.exists(video_path):
    print(f"Video file exists: {video_path}")
else:
    print(f"Error: Video file not found at {video_path}")

Video file exists: /content/livestock_detection/datasets/livestock_video_dataset(1).mp4


In [34]:
video_path = f'{livestock_count}/datasets/videodataset.mp4'

In [35]:
import cv2
from ultralytics import YOLO

# Load YOLOv8 model
def load_yolo_model():
    model = YOLO('yolov8n.pt')
    return model

# Process the video for object detection
def process_video(video_path):
    # Load the model
    model = load_yolo_model()

    # Open the video file
    cap = cv2.VideoCapture(video_path)
    results_folder = os.path.join(livestock_count, 'results')
    os.makedirs(results_folder, exist_ok=True)
    output_video_path = os.path.join(results_folder, 'detection_result.mp4')

    if not cap.isOpened():
        print(f"Error: Could not open video file {video_path}")
        return None
    else:
        print(f"Successfully opened video file: {video_path}")

        # Define codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

        # Process video frame by frame
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Perform object detection
            results = model(frame)

            # Get detected frame with bounding boxes
            result_frame = results[0].plot(show=False)

            # Write the frame to the output video
            out.write(result_frame)

        # Release everything after processing
        cap.release()
        out.release()

        print(f"Processed video saved to: {output_video_path}")
        return output_video_path

# Process the video and get the output path
if video_path:
    output_video_path = process_video(video_path)


Streaming output truncated to the last 5000 lines.
Speed: 1.5ms preprocess, 181.3ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 horse, 1 cow, 210.2ms
Speed: 1.5ms preprocess, 210.2ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 horse, 1 cow, 176.0ms
Speed: 3.8ms preprocess, 176.0ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 bench, 1 horse, 1 cow, 114.1ms
Speed: 1.9ms preprocess, 114.1ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 horse, 1 cow, 116.6ms
Speed: 2.0ms preprocess, 116.6ms inference, 1.2ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 horse, 1 cow, 114.2ms
Speed: 1.6ms preprocess, 114.2ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 dog, 1 cow, 117.0ms
Speed: 1.7ms preprocess, 117.0ms inference, 1.1ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 bench, 2 dogs, 1 cow, 121.

In [36]:
from google.colab import files

# Step 3: Download the processed video
def download_video(output_video_path):
    if output_video_path and os.path.exists(output_video_path):
        files.download(output_video_path)
    else:
        print("Error: Processed video file not found.")

# Download the video
if output_video_path:
    download_video(output_video_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Object detection for both (img and video) datasets


In [37]:
import os
import cv2
from ultralytics import YOLO
from google.colab import files
import shutil
import zipfile

# Define directory paths
livestock_count = '/content/livestock_detection'
datasets_folder = os.path.join(livestock_count, 'datasets')
results_folder = '/content/detected_datasets'

# Create necessary directories
os.makedirs(datasets_folder, exist_ok=True)
os.makedirs(results_folder, exist_ok=True)

# Upload and process files
def upload_and_process():
    uploaded = files.upload()
    for filename, content in uploaded.items():
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            file_path = os.path.join(datasets_folder, filename)
            with open(file_path, 'wb') as f:
                f.write(content)
            print(f"Saved {filename} to {file_path}")
        else:
            print(f"Unsupported file format: {filename}")

    # Process the uploaded images
    process_images()

# Load YOLOv8 model
model = YOLO('yolov8n.pt')

# Function to process images
def process_images():
    for root, _, files in os.walk(datasets_folder):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(root, file)
                print(f"Processing image: {file_path}")
                image = cv2.imread(file_path)
                results = model(image)
                result_image = results[0].plot(show=False)

                results_subfolder = os.path.join(results_folder, os.path.relpath(root, datasets_folder))
                os.makedirs(results_subfolder, exist_ok=True)

                result_image_path = os.path.join(results_subfolder, f'detected_{file}')
                cv2.imwrite(result_image_path, result_image)
                print(f"Result image saved to: {result_image_path}")

# Create a zip file with the results
def zip_results():
    zip_path = f"{results_folder}.zip"
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(results_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, results_folder)
                zipf.write(file_path, arcname)
    return zip_path

# Main execution
upload_and_process()

# Zip and provide download link for the results folder
zip_file = zip_results()
files.download(zip_file)
print(f"Download link for the results: {zip_file}")

Saving goat.jpg to goat (1).jpg
Saving camel.jpg to camel (1).jpg
Saving hipoimg.jpg to hipoimg (1).jpg
Saved goat (1).jpg to /content/livestock_detection/datasets/goat (1).jpg
Saved camel (1).jpg to /content/livestock_detection/datasets/camel (1).jpg
Saved hipoimg (1).jpg to /content/livestock_detection/datasets/hipoimg (1).jpg
Processing image: /content/livestock_detection/datasets/hipoimg (1).jpg

0: 480x640 1 bird, 2 cows, 4 elephants, 190.8ms
Speed: 4.6ms preprocess, 190.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
Result image saved to: /content/detected_datasets/./detected_hipoimg (1).jpg
Processing image: /content/livestock_detection/datasets/goat (1).jpg

0: 448x640 6 sheeps, 171.4ms
Speed: 5.1ms preprocess, 171.4ms inference, 1.1ms postprocess per image at shape (1, 3, 448, 640)
Result image saved to: /content/detected_datasets/./detected_goat (1).jpg
Processing image: /content/livestock_detection/datasets/camel (1).jpg

0: 448x640 3 horses, 158.9ms
Sp

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download link for the results: /content/detected_datasets.zip
